Задача:

- Обучить классификатор: продуктовый запрос vs. всё остальное (продуктовым можно считать запрос, который равен названию или описанию товара).
- Добавить логику поиска похожих товаров по продуктовому запросу.
- Вся логика должна быть завёрнута в метод get_answer(). Ответ на продуктовый запрос должен иметь вид "product_id title".


In [1]:
import os
import re
import string
import annoy
import codecs
import pickle

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec

import numpy as np
from tqdm.notebook import tqdm
import pandas as pd

In [2]:
DATADIR = 'data'
STORAGEDIR = 'storage'

In [3]:
# Загружаем датасет товаров
with codecs.open('/'.join([DATADIR, 'ProductsDataset.csv']), 'r', 'utf-8') as file:
    data = pd.read_csv(file)
data.head()    

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...


In [4]:
# Исправляем ошибки в наименовании колонки
data.rename(columns={'descrirption': 'description'}, inplace=True)

In [5]:
# Препроцессинг текста
exclude = set(string.punctuation)
stop_w = set(get_stop_words(language='ru'))
morpher = MorphAnalyzer()

In [6]:
# Компилируем шаблоны для быстрой обработки
# Спецсимволы и emoji
symbols_pattern = re.compile(pattern = "["
    u"\U0001F600-\U0001F64F"  # emoticons
    u"\U0001F300-\U0001F5FF"  # symbols & pictographs
    u"\U0001F680-\U0001F6FF"  # transport & map symbols
    u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
    "@_!#$%^&*()<>?/\|}{~:√•"
                       "]+", flags = re.UNICODE)
# Двойные пробелы
space_pattern = re.compile('\s+')

In [7]:
def clear_text(text):
    """ Функция удаления спецсимволов"""
    # Удаление спецсимволов и emoji
    pre = symbols_pattern.sub(r'',text)
    # Удаление двойных пробелов
    return space_pattern.sub(' ', pre)

In [8]:
def preprocess_text(text):
    """ Обработка текста """
    # srip + lower + punctuation
    sentence = ''.join([x for x in str(text).strip().lower() if x not in exclude])
    # Лемматизация и стопслова
    sentence = ' '.join([morpher.parse(word)[0].normal_form for word in sentence.split() if word not in stop_w])
    return clear_text(sentence)

In [9]:
# Обработка title
data['title'] = data['title'].apply(preprocess_text)
data.head()

,title,description,product_id,category_id,subcategory_id,properties,image_links
0,юбка детский orby,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
1,ботильон,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...
2,брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...
3,продать детский шапка,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
4,блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...


In [10]:
# Обработка description
data['description'] = data['description'].apply(preprocess_text)
data.head()

,title,description,product_id,category_id,subcategory_id,properties,image_links
0,юбка детский orby,новый носить раз реал красивый фото,58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...
1,ботильон,новыепривезти чехия указать размер 40ный малом...,5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...
2,брюки,размер 4042 брюки новый знать мерило покупка н...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...
3,продать детский шапка,продать шапкикажда 200ррозовый белый проданна,57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...
4,блузка,темносиний 42 размерсостояние отличноекак новы...,5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...


In [11]:
# Обучим модель word2vec

In [12]:
# Для word2vec возьмем title и description
sentences = []
for sentence in pd.Series(data['title'] + ' ' + data['description']).to_list():
    sentences.append(sentence.split())

In [13]:
sentences = [x for x in sentences if len(x)>2]
model_shop = Word2Vec(sentences=sentences, min_count=1, window=5)
model_shop.save('/'.join([STORAGEDIR, 'w2v_model_shop']))

In [14]:
def get_vector(sent, model):
    """ Функция возвращает вектор предложения """
    n_w2v = 0
    vector = np.zeros(100)
    for word in sent:
        if word in model.wv:
            vector += model.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector = vector / n_w2v
    return vector

In [16]:
# Строим индексы для продаж
index = annoy.AnnoyIndex(100, 'angular')

index_map = {}
counter = 0

# Упакуем в индекс 'title', 'description', 'product_id', чтобы всегда можно было вытянуть нужную информацию
for line in data[['title', 'description', 'product_id']].values:
    index_map[counter] = [line[0], line[1], line[2]]
    
    title = line[0].split()
    index.add_item(counter, get_vector(title, model_shop)) 
    counter += 1

index.build(10)
index.save('/'.join([STORAGEDIR, 'idx.ann']))
    

True

In [17]:
# Датасет вопросов с ответами

In [18]:
# Вопросы с ответами представим в виде словаря
questions = {}
senteties = []

with codecs.open('/'.join([DATADIR, 'Otvety.txt']), 'r', 'utf-8') as file:
    for line in tqdm(file):
        line = line.strip()
        # Предложения между ---, записываем в список. 
        # Получим, первый элемент - вопрос, последующие - ответы
        if line.startswith('---'):
            if len(senteties) > 1:
                try:
                    # Результат сохраняем в dict
                    question = senteties.pop(0) # Как мы помним, первый элемент - вопрос
                    questions[question] = senteties[0] # Сохраняем только первый ответ для экономии времени обработки
                    # Если данных слишком много, можно перенаправить запись в файл
                except Exception as err:
                    print(senteties, err)
            senteties = []
        else:
            senteties.append(line)

0it [00:00, ?it/s]

In [19]:
# Количество вопросов
len(questions)

1158597

In [20]:
# Сохраним результат (questions)
with open('/'.join([STORAGEDIR, 'questions.pkl']), 'wb') as file:
    pickle.dump(questions, file)

In [21]:
# Подготовим предложения для word2vec
sentences = []
count = 0
for k in tqdm(questions.items()):
    # Добавляем вопрос
    pre = preprocess_text(k)
    sentences.append(pre.split())

    count += 1
    # Для экономии ресурсов ограничимся 500000
    if count > 500000:
        break

  0%|          | 0/1158597 [00:00<?, ?it/s]

In [22]:
# Сохраним результат
with open('/'.join([STORAGEDIR, 'chat_questions_preprocessed.pkl']), 'wb') as file:
    pickle.dump(sentences, file)

In [23]:
# Обучим модель word2vec на наших вопросах
sentences = [i for i in sentences if len(i) > 2]
model_chat = Word2Vec(sentences=sentences, min_count=1, window=5)
model_chat.save('/'.join([STORAGEDIR, 'w2v_model_chat']))

In [24]:
# Формируем индекс для чата
index_chat = annoy.AnnoyIndex(100 ,'angular')

index_chat_map = {}
counter = 0

for k, v in tqdm(questions.items()):
    # Добавляем ответ
    index_chat_map[counter] = v
    
    question = preprocess_text(k)
    index_chat.add_item(counter, get_vector(question, model_chat))
    counter += 1

index_chat.build(10)
index_chat.save('/'.join([STORAGEDIR, 'speaker.ann']))


  0%|          | 0/1158597 [00:00<?, ?it/s]

True

In [25]:
# Classifications

In [26]:
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer

In [27]:
# Подготовим датасет с разметкой, состоящий из двух тематик (продажа товара и чат)

In [28]:
# Продажа, target = 1
df = pd.DataFrame(pd.concat([data['title'], data['description']], ignore_index=True))
df['target'] = 1
df.columns = ['data', 'target']

In [29]:
# Чат, target = 0

In [30]:
sentences_chat = []
count =0

for k in questions:
    sent = preprocess_text(k)
    sentences_chat.append(sent)
    
    count += 1
    if count > 500000:
        break

In [31]:
# Предложение более двух слов
sentences_chat = [sent for sent in sentences_chat if len(sent.split())>2]
chatdf = pd.DataFrame(sentences_chat)
chatdf['target'] = 0
chatdf.columns = ['data', 'target']

In [32]:
# Объединяем в один датасет
df = pd.concat([df, chatdf], ignore_index=True)

In [33]:
# Разбиваем на обучающую и валидационную выборки.
X_train, X_test, y_train, y_test = model_selection.train_test_split(df['data'], 
                                                                    df['target'], 
                                                                    test_size=0.3, 
                                                                    shuffle=True, 
                                                                    random_state=42)

In [34]:
# Выполним векторизация с помощью TfidfVectorizer
tfidf_vec = TfidfVectorizer().fit(X_train.values)
xtrain_tfidf = tfidf_vec.transform(X_train)
xvalid_tfidf = tfidf_vec.transform(X_test)

In [35]:
# Обучение модели
# Воспользуемся LogisticRegression
lr = linear_model.LogisticRegression()
lr.fit(xtrain_tfidf, y_train)

predictions = lr.predict(xvalid_tfidf)

C:\Users\Lyo\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [36]:
# Метрики
metrics.accuracy_score(y_test, predictions)

0.9840484083403854

In [37]:
# Если торговля, модель model_shop, если чат - model_chat.
def get_answer(question):
    """ Функция вовзращает ответ на вопрос, определяя тематику """
    question = preprocess_text(question).split()
    sent_vec = tfidf_vec.transform(question)
    if lr.predict(sent_vec)[0] == 1:
        model = model_shop
        print('Debug: model_shop')
        answer_index = index.get_nns_by_vector(get_vector(question, model), 1)
        # По условию возвращаем 'product_id' - [2]. Если потребуется 'description' - [1]
        answer = index_map[answer_index[0]][2]
    else:
        model = model_chat
        print('Debug: model_chat')
        answer_index = index_chat.get_nns_by_vector(get_vector(question, model), 1)
        answer = index_chat_map[answer_index[0]]
    return answer
            

In [38]:
assert(get_answer('Юбка детская ORBY').startswith('58e3cfe6132ca50e053f5f82'))

Debug: model_shop


In [39]:
assert(not get_answer('Где ключи от танка').startswith('5'))

Debug: model_chat


In [40]:
# Сохраняем модель
with open('/'.join([STORAGEDIR, 'model_lr.pkl']), 'wb') as file:
    pickle.dump(lr, file)

Вывод: В данном проекте мы решали задачу классификации. Улучшить метрику  можно путём обработки всех данных в датасете чата (метрика accuracy_score стремится к 100). Но уже сейчас можно дописать обёртку и получить виртуального ассистента.